Cifrado Simétrico — DES (Data Encryption Standard)

In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.9 MB/s eta 0:00:00


In [ ]:
import os, time, base64
import pandas as pd
from Crypto.Cipher import DES
from Crypto.Random import get_random_bytes



def generar_archivo(nombre, num_palabras):

    with open(nombre, "w") as f:
        f.write(("hola " * num_palabras).strip())


def leer_archivo(nombre):
    with open(nombre, "r") as f:
        return f.read()


def pad_data(data: bytes):
    padding_len = 8 - (len(data) % 8)
    padding = bytes([padding_len]) * padding_len
    return data + padding


def unpad_data(data: bytes):
    padding_len = data[-1]
    return data[:-padding_len]


def generar_clave_des():
    return get_random_bytes(8)  # DES usa una clave de 64 bits


def cifrar_des(key, mensaje_str):
    mensaje = mensaje_str.encode()
    padded = pad_data(mensaje)
    cipher = DES.new(key, DES.MODE_ECB)
    encrypted = cipher.encrypt(padded)
    return encrypted


def descifrar_des(key, encrypted):
    cipher = DES.new(key, DES.MODE_ECB)
    decrypted = cipher.decrypt(encrypted)
    unpadded = unpad_data(decrypted)
    return unpadded.decode()


tamaños = [10, 100, 1000, 10000, 100000, 1000000, 10000000]
resultados = []

for palabras in tamaños:
    print(f"\n Procesando archivo con {palabras} palabras...")

    nombre_archivo = f"text_{palabras}.txt"

    # -------- ETAPA 1: Leer archivo --------
    t1 = time.time()
    generar_archivo(nombre_archivo, palabras)
    texto = leer_archivo(nombre_archivo)
    t2 = time.time()

    # -------- ETAPA 2: Generar clave --------
    t3 = time.time()
    key = generar_clave_des()
    t4 = time.time()

    # -------- ETAPA 3: Cifrar --------
    t5 = time.time()
    ciphertext = cifrar_des(key, texto)
    t6 = time.time()

    # -------- ETAPA 4: Descifrar --------
    t7 = time.time()
    plaintext = descifrar_des(key, ciphertext)
    t8 = time.time()

    # -------- Guardar resultados --------
    resultados.append([
        "DES-ECB",
        palabras,
        len(texto),
        len(ciphertext),
        t2 - t1,   # T-E1 leer
        t4 - t3,   # T-E2 clave
        t6 - t5,   # T-E3 cifrar
        t8 - t7,   # T-E4 descifrar
        t8 - t1    # T-Total
    ])

df = pd.DataFrame(resultados, columns=[
    "Algoritmo",
    "#Palabras",
    "#Caracteres_Entrada",
    "#Caracteres_Salida",
    "T-E1 (Leer)",
    "T-E2 (Clave)",
    "T-E3 (Cifrar)",
    "T-E4 (Descifrar)",
    "T-Total",
])
print("\n=== TABLA RESUMEN ===")
print(df)

# Guardar CSV
df.to_csv("benchmark_results.csv", index=False)
print("\nCSV generado: benchmark_results.csv")



 Procesando archivo con 10 palabras...

 Procesando archivo con 100 palabras...

 Procesando archivo con 1000 palabras...

 Procesando archivo con 10000 palabras...

 Procesando archivo con 100000 palabras...

 Procesando archivo con 1000000 palabras...

 Procesando archivo con 10000000 palabras...

=== TABLA RESUMEN ===
  Algoritmo  #Palabras  #Caracteres_Entrada  #Caracteres_Salida  T-E1 (Leer)  \
0   DES-ECB         10                   49                  56     0.010659   
1   DES-ECB        100                  499                 504     0.000340   
2   DES-ECB       1000                 4999                5000     0.000192   
3   DES-ECB      10000                49999               50000     0.001904   
4   DES-ECB     100000               499999              500000     0.003702   
5   DES-ECB    1000000              4999999             5000000     0.017073   
6   DES-ECB   10000000             49999999            50000000     0.239851   

   T-E2 (Clave)  T-E3 (Cifrar)  T-E